In [1]:
import os
import sys
import pandas as pd
import tqdm
import numpy as np
from glob import glob

from process import *

---

In [5]:
# Process runoff spreadsheets given in path
variable  = 'AlturayCaudalInstantaneo'
region    = 7
raw_paths = glob(f'tmp/{variable}/{region}/*.xls') # <--- PATH
# raw_paths = glob(f'/home/lucas/PROYECTOS/En Proceso/A032445-Kimal Lo Aguirre 2/3_Aplicabilidad_PAS_156_Torres/2_Hidrologia/Datos Estaciones/Caudales/Rio Illapel en el Peral/*.xls')
data     = []
metadata = []
names    = []
for p in tqdm.tqdm(raw_paths, total=len(raw_paths)):
    df = pd.read_excel(p)
    ts, mdata = process_DGAtable(df, variable)
    data.append(ts['q_m3s'])
    names.append(ts.name)
    metadata.append(mdata)
data = pd.concat(data, keys=names).drop_duplicates().unstack().T.sort_index()
metadata = pd.concat(metadata, axis=1)
metadata = metadata.loc[:,~metadata.columns.duplicated()].copy().T

# metadata.to_csv(f'data/{variable}/{region}-Region_MetadataEstaciones.csv')
# data.to_csv(f'data/{variable}/{region}-Region_{variable}.csv')

  0%|          | 0/133 [00:00<?, ?it/s]

100%|██████████| 133/133 [00:19<00:00,  6.83it/s]


In [19]:
variable = 'PrecipitacionesMaximasEn24Horas'
# region   = 2
region = 12
raw_paths = glob(f'tmp/{variable}/{region}/*.xls')
data     = []
metadata = []
for p in tqdm.tqdm(raw_paths, total=len(raw_paths)):
    df = pd.read_excel(p)
    ts, mdata = process_DGAtable(df, variable)
    ts.index = ts.index.year
    data.append(ts)
    metadata.append(mdata)
    # del df, ts, mdata
data = pd.concat(data, axis=1)
metadata = pd.concat(metadata, axis=1)
metadata = metadata.loc[:,~metadata.columns.duplicated()].copy()

ncols    = []
mult_st  = []
for st in metadata.columns:
    x = data[st].dropna(how='all')
    if len(pd.DataFrame(x).columns)>1:
        ncol = [x.iloc[:,i].dropna() for i in range(len(x.columns))]
        ncols.append(pd.concat(ncol,axis=0).drop_duplicates())
        mult_st.append(st)
        del ncol
    del x
ncols = pd.concat(ncols, axis=1).reindex(data.index)
data  = pd.concat([data.drop(mult_st, axis=1), ncols], axis=1).sort_index()
metadata.T.to_csv(f'data/{variable}/{region}-Region_MetadataEstaciones.csv')
data.to_csv(f'data/{variable}/{region}-Region_{variable}.csv')

  0%|          | 0/72 [00:00<?, ?it/s]

100%|██████████| 72/72 [00:03<00:00, 22.06it/s]


In [22]:
def transform_coordinates(x):
    coord  = x.split(' ')
    num    = float(coord[0].replace('°',''))
    minute = float(coord[1].replace("'",""))/60
    second = float(coord[2].replace("''",""))/60/60
    return -(num+minute+second)

In [26]:
import geopandas as gpd
variable = 'PrecipitacionesMaximasEn24Horas'
metadata_paths = glob(f'data/{variable}/*Metadata*.csv')
data_paths     = glob(f'data/{variable}/*{variable}*.csv')

metadata = [pd.read_csv(m, index_col=0) for m in metadata_paths]
metadata = pd.concat(metadata, axis=0).drop_duplicates()
metadata['lon'] = metadata['lon'].map(lambda x: transform_coordinates(x))
metadata['lat'] = metadata['lat'].map(lambda x: transform_coordinates(x))
metadata = metadata[metadata.Cuenca != 'ISLAS DEL PACIFICO']

data = [pd.read_csv(m, index_col=0, parse_dates=False) for m in data_paths]
data = pd.concat(data, axis=1)
data = data[metadata.index]

metadata_shape = gpd.GeoDataFrame(geometry=gpd.points_from_xy(metadata.lon, metadata.lat),
                                  crs='epsg:4326').to_crs('epsg:32719')
metadata_shape.index = metadata.index

metadata['x'] = metadata_shape.geometry.x
metadata['y'] = metadata_shape.geometry.y
metadata_shape = pd.concat([metadata,metadata_shape], axis=1).set_geometry('geometry')

data.to_csv(f'data/DATA_{variable}.csv')
metadata.to_csv(f'data/MetadataEstaciones_{variable}.csv')
metadata_shape.to_file(f'data/MetadataEstaciones_{variable}.gpkg')